In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
!git clone https://github.com/jthickstun/watermark.git
import os
os.chdir("/kaggle/working/watermark/demo")
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import argparse
import torch
import random
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from mersenne import mersenne_rng
import pandas as pd
from tqdm import tqdm
import numpy as np  # Import numpy for array slicing
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Cloning into 'watermark'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 80 (delta 25), reused 67 (delta 14), pack-reused 0
Unpacking objects: 100% (80/80), 264.81 KiB | 3.89 MiB/s, done.
/kaggle/input/data_trunk.csv


In [2]:
!pip install sacremoses
import json
import torch.nn.functional as F

from transformers import AutoTokenizer, AutoModelForCausalLM, LogitsProcessorList
from transformers import MarianMTModel, MarianTokenizer

from collections import defaultdict

import numpy as np
import scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 10.5 MB/s eta 0:00:00a 0:00:01


# Generation

## Simulating Zero-Width Attack

In [3]:
# Zero width space
ZWSP = chr(0x200B)
# Zero width joiner
ZWJ = chr(0x200D)
# Zero width non-joiner
ZWNJ = chr(0x200C)

In [4]:
def inject_invisible_chars(text, invisible_chars=[ZWJ,ZWSP,ZWNJ], max_insertions=25):
    """
    Injects invisible characters into the text at random positions.

    Args:
      text: The original text to modify.
      invisible_chars: A list of Unicode characters with minimal visual presence.
      max_insertions: The maximum number of invisible characters to inject.

    Returns:
      A new string with invisible characters inserted at random positions.
    """
    import random

    candidate = list(text)  # Convert text to character list for modification
    for _ in range(max_insertions):
        # Choose a random invisible character
        invisible_char = random.choice(invisible_chars)
        # Choose a random insertion point within the text (avoiding the very end)
        insertion_point = random.randint(0, len(candidate) - 1)
        candidate.insert(insertion_point, invisible_char)
    return ''.join(candidate)

## Simulating Homoglyph Attack

In [5]:
def replace_with_homoglyph(char):
    """
    Replaces a character with a visually similar homoglyph.
    
    :param char: The character to replace.
    :return: The replaced character.
    """
    # Basic mapping of characters to homoglyphs
    homoglyph_mapping = {
        'a': 'а', # Latin 'a' to Cyrillic 'a'
        'e': 'е', # Latin 'e' to Cyrillic 'e'
        'o': 'о', # Latin 'o' to Cyrillic 'o'
        'i': 'і', # Latin 'i' to Cyrillic 'i'
        # Add more mappings as needed
    }
    
    return homoglyph_mapping.get(char, char) # Return the homoglyph if available, else return the original character

def homoglyph_attack(text):
    """
    Replaces characters in the text with their homoglyphs.
    
    :param text: The original text.
    :return: The text with homoglyphs.
    """
    # Replace each character in the text with its homoglyph
    homoglyph_text = ''.join(replace_with_homoglyph(c) for c in text)
    return homoglyph_text

## Simulating Bidirectional Reordering Attack

In [6]:
# Unicode Bidi override characters
PDF = chr(0x202C)
LRE = chr(0x202A)
RLE = chr(0x202B)
LRO = chr(0x202D)
RLO = chr(0x202E)

PDI = chr(0x2069)
LRI = chr(0x2066)
RLI = chr(0x2067)

In [7]:
class Swap():
    """Represents swapped elements in a string of text."""
    def __init__(self, one, two):
        self.one = one
        self.two = two
    
    def __repr__(self):
        return f"Swap({self.one}, {self.two})"

    def __eq__(self, other):
        return self.one == other.one and self.two == other.two

    def __hash__(self):
        return hash((self.one, self.two))

def some(*els):
    """Returns the arguments as a tuple with Nones removed."""
    return tuple(filter(None, tuple(els)))

def swaps(chars: str) -> set:
    """Generates all possible swaps for a string."""
    def pairs(chars, pre=(), suf=()):
        orders = set()
        for i in range(len(chars)-1):
            prefix = pre + tuple(chars[:i])
            suffix = suf + tuple(chars[i+2:])
            swap = Swap(chars[i+1], chars[i])
            pair = some(prefix, swap, suffix)
            orders.add(pair)
            orders.update(pairs(suffix, pre=some(prefix, swap)))
            orders.update(pairs(some(prefix, swap), suf=suffix))
        return orders
    return pairs(chars) | {tuple(chars)}

def unswap(el: tuple) -> str:
    """Reverts a tuple of swaps to the original string."""
    if isinstance(el, str):
        return el
    elif isinstance(el, Swap):
        return unswap((el.two, el.one))
    else:
        res = ""
        for e in el:
            res += unswap(e)
        return res

def uniswap(els):
    res = ""
    for el in els:
        if isinstance(el, Swap):
            res += uniswap([LRO, LRI, RLO, LRI, el.one, PDI, LRI, el.two, PDI, PDF, PDI, PDF])
        elif isinstance(el, str):
            res += el
        else:
            for subel in el:
                res += uniswap([subel])
    return res

def strings_to_file(file, string):
    with open(file, 'w') as f:
        for swap in swaps(string):
            uni = uniswap(swap)
            print(uni, file=f)

def print_strings(string):
    for swap in swaps(string):
        uni = uniswap(swap)
        print(uni)

In [8]:
def split_swap_merge(sentence,seed):
    """Splits a sentence into chunks of 5 to 10 characters, performs 
    a random swap on each chunk, and merges them back into one sentence."""
    chunks = []
    start_index = 0
    random.seed(seed)
    while start_index < len(sentence):
        chunk_size = random.randint(5, 10)  # Random chunk size between 5 and 10
        end_index = min(start_index + chunk_size, len(sentence))
        chunk = sentence[start_index:end_index]
        chunks.append(chunk)
        start_index = end_index
    swapped_chunks = []
    for chunk in chunks:
        if len(chunk) > 1:
            swaps_set = swaps(chunk)  # Generate all possible swaps for the chunk
            random_swap = random.choice(list(swaps_set))  # Choose a random swap
            swapped_chunks.append(uniswap(random_swap))  # Apply the random swap
        else:
            swapped_chunks.append(chunk)
    return "".join(swapped_chunks)  # Merge chunks back into a sentence (no spaces)

## Simulating Paraphrasing Attack

### Roundtrip Paraphrasing

In [9]:
def rt_translate(text):
    translation_language = "french"
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if translation_language == "french":
        en_ne_model_name = "Helsinki-NLP/opus-mt-tc-big-en-fr"
        en_ne_tokenizer = MarianTokenizer.from_pretrained(en_ne_model_name)
        en_ne_model = MarianMTModel.from_pretrained(en_ne_model_name).to(device)

        ne_en_model_name = "Helsinki-NLP/opus-mt-tc-big-fr-en"
        ne_en_tokenizer = MarianTokenizer.from_pretrained(ne_en_model_name)
        ne_en_model = MarianMTModel.from_pretrained(ne_en_model_name).to(device)
    elif translation_language == "russian":
        en_ne_model_name = "Helsinki-NLP/opus-mt-en-ru"
        en_ne_tokenizer = MarianTokenizer.from_pretrained(en_ne_model_name)
        en_ne_model = MarianMTModel.from_pretrained(en_ne_model_name).to(device)

        ne_en_model_name = "Helsinki-NLP/opus-mt-ru-en"
        ne_en_tokenizer = MarianTokenizer.from_pretrained(ne_en_model_name)
        ne_en_model = MarianMTModel.from_pretrained(ne_en_model_name).to(device)
    else:
        raise
    try:
        tokens = en_ne_tokenizer(text.split('. '), return_tensors="pt", padding=True).to(device)
        tokens = en_ne_model.generate(**tokens,max_new_tokens=52)
        french_text = ' '.join([en_ne_tokenizer.decode(t, skip_special_tokens=True) for t in tokens])

        tokens = ne_en_tokenizer(french_text.split('. '), return_tensors="pt", padding=True).to(device)
        tokens = ne_en_model.generate(**tokens,max_new_tokens=512)
        roundtrip_text = ' '.join([ne_en_tokenizer.decode(t, skip_special_tokens=True) for t in tokens])
    except:
        roundtrip_text = ""
    return roundtrip_text

### T5-based Paraphrasing

In [10]:

def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=1,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=512
):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
    model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids.to(device)
 
    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )
 
    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return res

## Generate Watermarked Text 

In [11]:
def generate_shift(model, prompt, vocab_size, n, m, key):
    """
    m is the requested length of the generated text
    n is the length of the watermark sequence
    key is a key for generating the random watermark sequence
    """
    rng = mersenne_rng(key) # used to generate a pseudorandom key
    xi = torch.tensor([rng.rand() for _ in range(n*vocab_size)]).view(n, vocab_size) 
    shift = torch.randint(n, (1,))

    inputs = prompt.to(model.device)
    attn = torch.ones_like(inputs)
    past = None
    for i in range(m):
        with torch.no_grad():
            if past:
                output = model(inputs[:, -1:], past_key_values=past, attention_mask=attn)
            else:
                output = model(inputs)

        probs = torch.nn.functional.softmax(output.logits[:, -1, :vocab_size], dim=-1).cpu()
        token = exp_sampling(probs, xi[(shift+i)%n, :]).to(model.device)
        inputs = torch.cat([inputs, token], dim=-1)

        past = output.past_key_values
        attn = torch.cat([attn, attn.new_ones((attn.shape[0], 1))], dim=-1)

    return inputs.detach().cpu()

def exp_sampling(probs, u):
    return torch.argmax(u ** (1/probs), axis=1).unsqueeze(-1)


In [20]:
def main():
    torch.manual_seed(0)
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model_name = "facebook/opt-350m"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
    prompt = input("Insert a prompt:\n")
    tokens = tokenizer.encode(prompt, return_tensors='pt', truncation=True, max_length=2048).to(device)
    watermarked_tokens = generate_shift(model, tokens, len(tokenizer), 256, 24, 42)[0]
    watermarked_text = tokenizer.decode(watermarked_tokens, skip_special_tokens=True)
    print("Unaltered Output:\n")
    print(watermarked_text)
    print("Executing Attacks...")
    max_insertions = len(watermarked_text)//5
    altered_text = inject_invisible_chars(watermarked_text, max_insertions=max_insertions)
    altered_text = homoglyph_attack(altered_text)
    bidi_flag = input("Would you like to execute a Bidirectional Reordering Attack?\nYes/No")
    bidi_flag = True if bidi_flag.lower()=="yes" or bidi_flag.lower()=="y" else False 
    if bidi_flag:
        altered_text = split_swap_merge(altered_text,10)
    print("Altered Output:")
    print(altered_text)
    paraphrasing_flag = input("Would you like to execute a Paraphrasing Attack?\nYes/No")
    paraphrasing_flag = True if paraphrasing_flag.lower()=="yes" or paraphrasing_flag.lower()=="y" else False 
    if paraphrasing_flag:
        paraphrasing_type_flag = int(input("Choose one:\n1. Human-like paraphrasing\n2. Machine learning powered paraphrasing\n"))
        if paraphrasing_type_flag == 1:
            print(rt_translate(watermarked_text))
        else:
            print(paraphrase(watermarked_text)[0])

if __name__ == '__main__':
    main()


Insert a prompt:
 A good thesis is


Unaltered Output:

A good thesis is usually one where you build upon the concepts you learned in the last year and revise and proofread the main portions of your
Executing Attacks...


Would you like to execute a Bidirectional Reordering Attack?
Yes/No yes


Altered Output:
‭⁦‮⁦‭⁦‮⁦t⁩⁦‌⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦ ⁩⁦d⁩‬⁩‬⁩⁦A‭⁦‮⁦g⁩⁦ ⁩‬⁩‬‭⁦‮⁦о⁩⁦о⁩‬⁩‬⁩‬⁩‬⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦s⁩⁦і⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦s⁩⁦е⁩‬⁩‬⁩⁦h⁩‬⁩‬⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦u⁩⁦​⁩‬⁩‬⁩⁦‭⁦‮⁦і⁩⁦ ⁩‬⁩‬s ‭⁦‮⁦s⁩⁦u⁩‬⁩‬⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦о⁩⁦ ⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦l⁩⁦l⁩‬⁩‬⁩⁦а⁩‬⁩‬y⁩‬⁩‬‭⁦‮⁦n⁩⁦​⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦y⁩⁦ ⁩‬⁩‬⁩⁦е‭⁦‮⁦w⁩⁦ ⁩‬⁩‬h‭⁦‮⁦r⁩⁦е⁩‬⁩‬е⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦b⁩⁦‍⁩‬⁩‬⁩⁦о‭⁦‮⁦ ⁩⁦u⁩‬⁩‬⁩‬⁩‬u‭⁦‮⁦l⁩⁦і⁩‬⁩‬d‭⁦‮⁦u⁩⁦ ⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦h⁩⁦t⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦ ⁩⁦​⁩‬⁩‬⁩⁦‌pоn⁩‬⁩‬⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦​⁩⁦о⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦ ⁩⁦е⁩‬⁩‬⁩⁦​⁩‬⁩‬c⁩‬⁩‬‭⁦‮⁦c⁩⁦n⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦s⁩⁦t⁩‬⁩‬⁩⁦‭⁦‮⁦p⁩⁦е⁩‬⁩‬⁩‬⁩‬ ‭⁦‮⁦о⁩⁦y⁩‬⁩‬‭⁦‮⁦ ⁩⁦u⁩‬⁩‬​‭⁦‮⁦е⁩⁦l⁩‬⁩‬а‭⁦‮⁦r⁩⁦​⁩‬⁩‬‭⁦‮⁦е⁩⁦n⁩‬⁩‬d і‭⁦‮⁦n⁩⁦‌⁩‬⁩‬​ ‭⁦‮⁦‍⁩⁦​⁩‬⁩‬‭⁦‮⁦h⁩⁦t⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦ ⁩⁦t⁩‬⁩‬⁩⁦е​‭⁦‮⁦​⁩⁦ ⁩‬⁩‬l‭⁦‮⁦s⁩⁦а⁩‬⁩‬⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦n⁩⁦а⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦ ⁩⁦r⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦а⁩⁦е⁩‬⁩‬⁩⁦y⁩‬⁩‬⁩‬⁩‬⁩‬⁩‬‍‭⁦‮⁦‭⁦‮⁦‌⁩⁦ ⁩‬⁩‬⁩⁦d⁩‬⁩‬‭⁦‮⁦е⁩⁦r⁩‬⁩‬‍v​‌і​sе​‭⁦‮⁦а⁩⁦ ⁩‬⁩‬nd‭⁦‮⁦‭⁦‮⁦r⁩⁦​⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦f⁩⁦о⁩‬⁩‬⁩⁦‭⁦‮⁦p⁩⁦ ⁩‬⁩‬rо​⁩‬⁩‬⁩‬⁩‬е‭⁦‮⁦‭⁦‮⁦е⁩⁦h⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦ ⁩⁦d⁩‬⁩‬⁩⁦а⁩‬⁩‬t⁩‬⁩‬‍‭⁦‮⁦‭⁦‮⁦і⁩⁦а⁩‬⁩‬⁩⁦ ‭⁦‮⁦‌⁩⁦m⁩‬⁩‬⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦о⁩⁦і⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦r⁩⁦о⁩‬⁩‬⁩⁦‭⁦‮⁦ ⁩⁦n⁩‬⁩‬p⁩‬⁩‬t⁩‬⁩‬‭⁦‮⁦s⁩⁦n⁩‬⁩‬ ‭⁦‮⁦​⁩⁦о⁩‬⁩‬f‭⁦‮⁦‭⁦‮⁦r

Would you like to execute a Paraphrasing Attack?
Yes/No no


# Detection

## Detection functions

In [13]:
!git clone https://github.com/jthickstun/watermark.git

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Cloning into 'watermark'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 80 (delta 25), reused 67 (delta 14), pack-reused 0
Unpacking objects: 100% (80/80), 264.81 KiB | 3.78 MiB/s, done.


In [14]:
import os
os.chdir("/kaggle/working/watermark/demo")

In [15]:
import os, sys, argparse, time

import numpy as np
from transformers import AutoTokenizer
from mersenne import mersenne_rng

import pyximport
pyximport.install(reload_support=True, language_level=sys.version_info[0],
                  setup_args={'include_dirs':np.get_include()})
from levenshtein import levenshtein

def permutation_test(tokens,key,n,k,vocab_size,n_runs=100):
    rng = mersenne_rng(key)
    xi = np.array([rng.rand() for _ in range(n*vocab_size)], dtype=np.float32).reshape(n,vocab_size)
    test_result = detect(tokens,n,k,xi)

    p_val = 0
    for run in range(n_runs):
        xi_alternative = np.random.rand(n, vocab_size).astype(np.float32)
        null_result = detect(tokens,n,k,xi_alternative)

        # assuming lower test values indicate presence of watermark
        p_val += null_result <= test_result

    return (p_val+1.0)/(n_runs+1.0)


def detect(tokens,n,k,xi,gamma=0.0):
    m = len(tokens)
    n = len(xi)

    A = np.empty((m-(k-1),n))
    for i in range(m-(k-1)):
        for j in range(n):
            A[i][j] = levenshtein(tokens[i:i+k],xi[(j+np.arange(k))%n],gamma)

    return np.min(A)


In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1929,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from /root/.pyxbld/temp.linux-x86_64-cpython-310/kaggle/working/watermark/demo/levenshtein.c:1205:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~


## Counteracting Zero-width Attacks

In [16]:
# Zero width space
ZWSP = chr(0x200B)
# Zero width joiner
ZWJ = chr(0x200D)
# Zero width non-joiner
ZWNJ = chr(0x200C)
# Zero width word joiner
ZWWJ = chr(0x2060)
# Zero width no-break space
ZWNBSP = chr(0xFEFF)

def replace_invisible_chars(text):
    """
    Finds the positions of invisible characters in the text and replaces them.

    Args:
      text: The text to check for invisible characters.

    Returns:
      A list of indices where invisible characters are found.
    """
    invisible_chars = [ZWSP, ZWJ, ZWNJ, ZWWJ, ZWNBSP]
    cleaned_text = text
    for char in invisible_chars:
        # Create a new string with replacements
        cleaned_text = cleaned_text.replace(char, "")
    return cleaned_text


## Counteracting Homoglyph Attacks

In [17]:
def normalize_homoglyph(text,i,char):
    """
    Replaces a character with a visually similar homoglyph.
    
    :param char: The character to replace.
    :return: The replaced character.
    """
    # Basic mapping of characters to homoglyphs
    normal_to_homoglyph = {
        'а':'a', #  Cyrillic 'a' to Latin 'a'
        'е':'e' , # Cyrillic 'e' to Latin 'e'
        'о':'o' , # Cyrillic 'o' to Latin 'o'
        'і':'i' , # Cyrillic 'i' to Latin 'i' 
    }
    return normal_to_homoglyph.get(char, char) # Return the homoglyph if available, else return the original character

def counteract_homoglyph(text):
    """
    Replaces homoglyphs in the text with their normalized counterparts.
    
    :param text: The original text with homoglyph.
    :return: The text with homoglyphs.
    """
    # Replace each character in the text with its homoglyph
    normalized_text = ''.join(normalize_homoglyph(text,i,c) for i,c in enumerate(text))
    return normalized_text

## Flagging Bidirectional Reordering 

In [18]:
def has_right_to_left_letters(text):
    """
    Checks if the text contains any letters written in right-to-left (RTL) languages.

    Args:
    text: The text to check.

    Returns:
    True if the text contains any RTL letters, False otherwise.
    """

    # Define a dictionary for RTL script Unicode ranges
    # based on https://www.unicode.org/charts/ and https://en.wikipedia.org/wiki/Right-to-left_script
    # Added Current scripts only
    rtl_ranges = {
      "Arabic": (0x0600, 0x06FF),
      "Hebrew": (0x0590, 0x05FF),
      "Thaana": (0x0780, 0x07BF),
      "Syriac": (0x0700, 0x074F),
      "Mandaic": (0x0840, 0x085F),
      "Samaritan": (0x0800, 0x083F),
      "Mende Kikakui": (0x1E800, 0x1E8DF),
      "NKo": (0x07C0, 0x07FF),
      "Garay": (0x10D40, 0x10D8F),
      "Adlam": (0x1E900, 0x1E95F),
      "Hanifi Rohingya": (0x10D00, 0x10D3F),
      "Yezidi": (0x10E80, 0x10EBF)
  }

    for char in text:
        for script, range in rtl_ranges.items():
            if range[0] <= ord(char) <= range[1]:
                return True
    return False

def is_bidi_manipulated(text):
    """
    Checks if a text (assumed to be non-RTL) contains BiDi override characters,
    potentially indicating manipulation.

    Args:
        text: The text to check.

    Returns:
        A tuple containing two elements:
            - is_manipulated (bool): True if BiDi override characters are found, 
                                     False otherwise.
            - override_count (int): The number of BiDi override characters found.
    """

    # Define a set of BiDi override characters
    bidi_overrides = {
        chr(0x202A): "LRE (Left-to-Right Embedding)",
        chr(0x202B): "RLE (Right-to-Left Embedding)",
        chr(0x202C): "PDF (Pop Directional Formatting)",
        chr(0x202D): "LRO (Left-to-Right Override)",
        chr(0x202E): "RLO (Right-to-Left Override)",
        chr(0x2066): "LRI (Left-to-Right Isolate)",
        chr(0x2067): "RLI (Right-to-Left Isolate)",
        chr(0x2069): "PDI (Pop Directional Isolate)"
    }

    # Check if the text is not empty and assumed to be non-RTL
    if not text or has_right_to_left_letters(text):  
        return False, 0

    override_count = 0
    for char in text:
        if char in bidi_overrides:
            override_count += 1

    return override_count > 0, override_count

## Detection Pipeline with Counteracting

In [21]:
from tqdm import tqdm
import pandas as pd

def main():
    tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")
    text = input("Enter Candidate Text:\n")
    cleaned_text_zw = replace_invisible_chars(text)
    if cleaned_text_zw != text:
        print("Zero-width attack effect undone.")
    cleaned_text_homoglyph = counteract_homoglyph(cleaned_text_zw)
    if cleaned_text_zw != cleaned_text_homoglyph:
        print("Homoglyph attack effect undone.")
    
    bidi_flag,reordering_count = is_bidi_manipulated(cleaned_text_homoglyph)
    if bidi_flag:
        print(f'This text is likely manipulated using Bidirectional Reordering characters.\nThere are {reordering_count} reordering character(s) in the text.\nResults: "Inconclusive".\n')
    else:
        tokens = tokenizer.encode(cleaned_text_homoglyph, return_tensors='pt', truncation=True, max_length=2048).numpy()[0]
        t0 = time.time()
        pval = permutation_test(tokens,42,256,len(tokens),len(tokenizer))
        label = "watermarked" if pval <= 0.01 else "unwatermarked"
        if label == "watermarked":
            print("This text is Machine-Generated.")
        elif label == "unwatermarked":
            print("This text is Human-Written.")
        print(f"p-value: {pval:.4f}, label: {label} (elapsed time: {time.time()-t0:.2f}s)")

if __name__ == '__main__':
    main()

Enter Candidate Text:
 ‭⁦‮⁦‭⁦‮⁦t⁩⁦‌⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦ ⁩⁦d⁩‬⁩‬⁩⁦A‭⁦‮⁦g⁩⁦ ⁩‬⁩‬‭⁦‮⁦о⁩⁦о⁩‬⁩‬⁩‬⁩‬⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦s⁩⁦і⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦s⁩⁦е⁩‬⁩‬⁩⁦h⁩‬⁩‬⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦u⁩⁦​⁩‬⁩‬⁩⁦‭⁦‮⁦і⁩⁦ ⁩‬⁩‬s ‭⁦‮⁦s⁩⁦u⁩‬⁩‬⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦о⁩⁦ ⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦l⁩⁦l⁩‬⁩‬⁩⁦а⁩‬⁩‬y⁩‬⁩‬‭⁦‮⁦n⁩⁦​⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦y⁩⁦ ⁩‬⁩‬⁩⁦е‭⁦‮⁦w⁩⁦ ⁩‬⁩‬h‭⁦‮⁦r⁩⁦е⁩‬⁩‬е⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦b⁩⁦‍⁩‬⁩‬⁩⁦о‭⁦‮⁦ ⁩⁦u⁩‬⁩‬⁩‬⁩‬u‭⁦‮⁦l⁩⁦і⁩‬⁩‬d‭⁦‮⁦u⁩⁦ ⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦h⁩⁦t⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦ ⁩⁦​⁩‬⁩‬⁩⁦‌pоn⁩‬⁩‬⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦​⁩⁦о⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦ ⁩⁦е⁩‬⁩‬⁩⁦​⁩‬⁩‬c⁩‬⁩‬‭⁦‮⁦c⁩⁦n⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦s⁩⁦t⁩‬⁩‬⁩⁦‭⁦‮⁦p⁩⁦е⁩‬⁩‬⁩‬⁩‬ ‭⁦‮⁦о⁩⁦y⁩‬⁩‬‭⁦‮⁦ ⁩⁦u⁩‬⁩‬​‭⁦‮⁦е⁩⁦l⁩‬⁩‬а‭⁦‮⁦r⁩⁦​⁩‬⁩‬‭⁦‮⁦е⁩⁦n⁩‬⁩‬d і‭⁦‮⁦n⁩⁦‌⁩‬⁩‬​ ‭⁦‮⁦‍⁩⁦​⁩‬⁩‬‭⁦‮⁦h⁩⁦t⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦ ⁩⁦t⁩‬⁩‬⁩⁦е​‭⁦‮⁦​⁩⁦ ⁩‬⁩‬l‭⁦‮⁦s⁩⁦а⁩‬⁩‬⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦n⁩⁦а⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦ ⁩⁦r⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦а⁩⁦е⁩‬⁩‬⁩⁦y⁩‬⁩‬⁩‬⁩‬⁩‬⁩‬‍‭⁦‮⁦‭⁦‮⁦‌⁩⁦ ⁩‬⁩‬⁩⁦d⁩‬⁩‬‭⁦‮⁦е⁩⁦r⁩‬⁩‬‍v​‌і​sе​‭⁦‮⁦а⁩⁦ ⁩‬⁩‬nd‭⁦‮⁦‭⁦‮⁦r⁩⁦​⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦f⁩⁦о⁩‬⁩‬⁩⁦‭⁦‮⁦p⁩⁦ ⁩‬⁩‬rо​⁩‬⁩‬⁩‬⁩‬е‭⁦‮⁦‭⁦‮⁦е⁩⁦h⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦ ⁩⁦d⁩‬⁩‬⁩⁦а⁩‬⁩‬t⁩‬⁩‬‍‭⁦‮⁦‭⁦‮⁦і⁩⁦а⁩‬⁩‬⁩⁦ ‭⁦‮⁦‌⁩⁦m⁩‬⁩‬⁩‬⁩‬‭⁦‮⁦‭⁦‮⁦о⁩⁦і⁩‬⁩‬⁩⁦‭⁦‮⁦‭⁦‮⁦r⁩⁦о⁩‬⁩‬⁩⁦‭⁦‮⁦ ⁩⁦n⁩‬⁩‬p⁩‬⁩‬t⁩‬⁩‬‭⁦‮⁦s⁩⁦n⁩‬⁩‬ ‭⁦‮⁦​⁩⁦о⁩‬⁩‬f‭⁦

Zero-width attack effect undone.
Homoglyph attack effect undone.
This text is likely manipulated using Bidirectional Reordering characters.
There are 840 reordering character(s) in the text.
Results: "Inconclusive".

